# Lab 7 Neural Language Model
A language model predicts the next word in the sequence based on the specific words that have come before it in the sequence.

It is also possible to develop language models at the character level using neural networks. The benefit of character-based language models is their small vocabulary and flexibility in handling any words, punctuation, and other document structure. This comes at the cost of requiring larger models that are slower to train.

Nevertheless, in the field of neural language models, character-based models offer a lot of promise for a general, flexible and powerful approach to language modeling.

As a prerequisite for the lab, make sure to pip install:
- keras
- tensorflow
- h5py

# Source Text Creation

To start out with, we'll be using a simple nursery rhyme. It's quite short so we can actually train something on your CPU and see relatively interesting results. Please copy and paste the following text in a text file and save it as "rhyme.txt". Place this in the same directory as this jupyter notebook:

In [1]:
!pip install tensorflow
!pip install keras
!pip install h5py

In [2]:
s='Sing a song of sixpence,\
A pocket full of rye.\
Four and twenty blackbirds,\
Baked in a pie.\
When the pie was opened\
The birds began to sing;\
Wasn’t that a dainty dish,\
To set before the king.\
The king was in his counting house,\
Counting out his money;\
The queen was in the parlour,\
Eating bread and honey.\
The maid was in the garden,\
Hanging out the clothes,\
When down came a blackbird\
And pecked off her nose.'

with open('rhymes.txt','w') as f:
  f.write(s)

# Sequence Generation

A language model must be trained on the text, and in the case of a character-based language model, the input and output sequences must be characters.

The number of characters used as input will also define the number of characters that will need to be provided to the model in order to elicit the first predicted character.

After the first character has been generated, it can be appended to the input sequence and used as input for the model to generate the next character.

Longer sequences offer more context for the model to learn what character to output next but take longer to train and impose more burden on seeding the model when generating text.

We will use an arbitrary length of 10 characters for this model.

There is not a lot of text, and 10 characters is a few words.

We can now transform the raw text into a form that our model can learn; specifically, input and output sequences of characters.

In [3]:
#load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [4]:
#load text
raw_text = load_doc('rhymes.txt')
print(raw_text)

# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)

# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):
    # select sequence of tokens
    seq = raw_text[i-length:i+1]
    # store
    sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

Sing a song of sixpence,A pocket full of rye.Four and twenty blackbirds,Baked in a pie.When the pie was openedThe birds began to sing;Wasn’t that a dainty dish,To set before the king.The king was in his counting house,Counting out his money;The queen was in the parlour,Eating bread and honey.The maid was in the garden,Hanging out the clothes,When down came a blackbirdAnd pecked off her nose.
Total Sequences: 384


In [5]:
# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

# Train a Model
In this section, we will develop a neural language model for the prepared sequence data.

The model will read encoded characters and predict the next character in the sequence. A Long Short-Term Memory recurrent neural network hidden layer will be used to learn the context from the input sequence in order to make the predictions.

In [6]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [113]:
# load

in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

The sequences of characters must be encoded as integers.This means that each unique character will be assigned a specific integer value and each sequence of characters will be encoded as a sequence of integers. We can create the mapping given a sorted set of unique characters in the raw input data. The mapping is a dictionary of character values to integer values.

Next, we can process each sequence of characters one at a time and use the dictionary mapping to look up the integer value for each character. The result is a list of integer lists.

We need to know the size of the vocabulary later. We can retrieve this as the size of the dictionary mapping.

In [114]:
# integer encode sequences of characters
chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))
sequences = list()
for line in lines:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    sequences.append(encoded_seq)

# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)

Vocabulary Size: 38


The model is defined with an input layer that takes sequences that have 10 time steps and 38 features for the one hot encoded input sequences. Rather than specify these numbers, we use the second and third dimensions on the X input data. This is so that if we change the length of the sequences or size of the vocabulary, we do not need to change the model definition.

The model has a single LSTM hidden layer with 75 memory cells. The model has a fully connected output layer that outputs one vector with a probability distribution across all characters in the vocabulary. A softmax activation function is used on the output layer to ensure the output has the properties of a probability distribution.

The model is learning a multi-class classification problem, therefore we use the categorical log loss intended for this type of problem. The efficient Adam implementation of gradient descent is used to optimize the model and accuracy is reported at the end of each batch update. The model is fit for 50 training epochs.

# To Do:
- Try different numbers of memory cells
- Try different types and amounts of recurrent and fully connected layers
- Try different lengths of training epochs
- Try different sequence lengths and pre-processing of data
- Try regularization techniques such as Dropout

In [9]:
# define model
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history=model.fit(X, y, epochs=50)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 75)                34200     
_________________________________________________________________
dense (Dense)                (None, 38)                2888      
Total params: 37,088
Trainable params: 37,088
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
12/12 [==============================] - 0s 6ms/step - loss: 3.6113 - accuracy: 0.1042
Epoch 2/50
12/12 [==============================] - 0s 5ms/step - loss: 3.5190 - accuracy: 0.1615
Epoch 3/50
12/12 [==============================] - 0s 5ms/step - loss: 3.2327 - accuracy: 0.1589
Epoch 4/50
12/12 [==============================] - 0s 6ms/step - loss: 3.1128 - accuracy: 0.1615
Epoch 5/50
12/12 [==============================] - 0s 6ms/step - loss: 3.0642 - accuracy: 0.1745
Epoch 6/50
12/12 [=

In [115]:
# save the model to file
model.save('model.h5')
# save the mapping
dump(mapping, open('mapping.pkl', 'wb'))

In [15]:
X.shape

(384, 10, 38)

# Generating Text

We must provide sequences of 10 characters as input to the model in order to start the generation process. We will pick these manually. A given input sequence will need to be prepared in the same way as preparing the training data for the model. 

In [13]:
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        # predict character
        # yhat = model.predict_classes(encoded, verbose=0)
        yhat = np.argmax(model.predict(encoded), axis=-1)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

# load the model
model = load_model('model.h5')
# load the mapping
mapping = load(open('mapping.pkl', 'rb'))

Running the example generates three sequences of text.

The first is a test to see how the model does at starting from the beginning of the rhyme. The second is a test to see how well it does at beginning in the middle of a line. The final example is a test to see how well it does with a sequence of characters never seen before.

In [14]:
# test start of rhyme
print(generate_seq(model, mapping, 10, 'Sing a son', 20))
# test mid-line
print(generate_seq(model, mapping, 10, 'king was i', 20))
# test not in original
print(generate_seq(model, mapping, 10, 'hello worl', 20))

Sing a song onf out of e fe of
king was in hee hoe ee  oue  o
hello worl.,Fe a ngin oous hhe


    Sing a song of sixpence,
    A pocket full of rye.
    Four and twenty blackbirds,
    Baked in a pie.

    When the pie was opened
    The birds began to sing;
    Wasn’t that a dainty dish,
    To set before the king.

    The king was in his counting house,
    Counting out his money;
    The queen was in the parlour,
    Eating bread and honey.

    The maid was in the garden,
    Hanging out the clothes,
    When down came a blackbird
    And pecked off her nose.

If the results aren't satisfactory, try out the suggestions above or these below:
- Padding. Update the example to provides sequences line by line only and use padding to fill out each sequence to the maximum line length.
- Sequence Length. Experiment with different sequence lengths and see how they impact the behavior of the model.
- Tune Model. Experiment with different model configurations, such as the number of memory cells and epochs, and try to develop a better model for fewer resources.


# Deliverables to receive credit

1. (4 points) Optimize the cells above to tune the model so that it generates text that closely resembles the orginal line from the rhyme, or at least generates sensible words. It's okay if the third example using unseen text still looks somewhat strange  though. Again, this is a toy problem, as language models require a lot of computation. This toy problem is great for rapid experimentation to explore different aspects of deep learning language models.
2. (3 points) Write a function to split the text corpus file into training and validation and pipe the validation data into the model.fit() function to be able to track validation error per epoch. Lookup Keras documentation to see how this is handled.
3. (3 points) Write a summary (methods and results) in the cells below of the different things you applied. You must include your intuitions behind what did work and what did not work well.
4. (Extra Credit 2.5 points) Do something even more interesting. Try a different source text. Train a word-level model. We'll leave it up to your creativity to explore and write a summary of your methods and results.


In [116]:
# 1.1 Define model 1 with 100 memory cells in the single LSTM hidden layer, and 100 training epoches
model1 = Sequential()
model1.add(LSTM(100, input_shape=(X.shape[1], X.shape[2])))
model1.add(Dense(vocab_size, activation='softmax'))
print(model1.summary())
# compile model
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history=model1.fit(X, y, epochs=100)

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_40 (LSTM)               (None, 100)               55600     
_________________________________________________________________
dense_31 (Dense)             (None, 38)                3838      
Total params: 59,438
Trainable params: 59,438
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
12/12 [==============================] - 0s 7ms/step - loss: 3.6055 - accuracy: 0.0990
Epoch 2/100
12/12 [==============================] - 0s 7ms/step - loss: 3.4548 - accuracy: 0.1432
Epoch 3/100
12/12 [==============================] - 0s 7ms/step - loss: 3.1565 - accuracy: 0.1302
Epoch 4/100
12/12 [==============================] - 0s 7ms/step - loss: 3.0752 - accuracy: 0.1589
Epoch 5/100
12/12 [==============================] - 0s 7ms/step - loss: 3.0515 - accuracy: 0.1589
Epoch 6/100

In [117]:
# save the model to file
model1.save('model1.h5')

In [118]:
# test start of rhyme
print(generate_seq(model1, mapping, 10, 'Sing a son', 20))
# test mid-line
print(generate_seq(model1, mapping, 10, 'king was i', 20))
# test not in original
print(generate_seq(model1, mapping, 10, 'hello worl', 20))


Sing a song of sixpence,A pock
king was in his counting house
hello worl,,oueddgtf  en tt lf


**The first model has very high accuracy. The prediction on the first and second test examples look great. The third example doesn't look good, but it's ok**

In [119]:
# 1.2 Define model 2, adding on model 1 with one dropout layer between input and output layers.
from keras.layers import Dropout

model2 = Sequential()
model2.add(LSTM(100, input_shape=(X.shape[1], X.shape[2])))
model2.add(Dropout(rate=0.2))
model2.add(Dense(vocab_size, activation='softmax'))
print(model2.summary())
# compile model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history=model2.fit(X, y, epochs=100)

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_41 (LSTM)               (None, 100)               55600     
_________________________________________________________________
dropout_18 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 38)                3838      
Total params: 59,438
Trainable params: 59,438
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
12/12 [==============================] - 0s 8ms/step - loss: 3.6103 - accuracy: 0.0885
Epoch 2/100
12/12 [==============================] - 0s 7ms/step - loss: 3.4939 - accuracy: 0.1615
Epoch 3/100
12/12 [==============================] - 0s 7ms/step - loss: 3.2364 - accuracy: 0.1589
Epoch 4/100
12/12 [==============================] - 0s 8ms/step - loss: 3.10

In [120]:
# save the model to file
model2.save('model2.h5')

In [121]:
# test start of rhyme
print(generate_seq(model2, mapping, 10, 'Sing a son', 20))
# test mid-line
print(generate_seq(model2, mapping, 10, 'king was i', 20))
# test not in original
print(generate_seq(model2, mapping, 10, 'hello worl', 20))


Sing a song of sixpence,A pock
king was in his counting house
hello worl,.Fo  krt. oulllltho


**The second model is also performing pretty well for the first two test cases. The third case is still not good, but it's ok. I added the dropout layer trying to regulate overfitting by dropping out some data during training.**

In [128]:
# 1.3 Define model 3, adding two hidden layers of fully-connected Dense layers
model3 = Sequential()
model3.add(LSTM(100, input_shape=(X.shape[1], X.shape[2])))
model3.add(Dropout(rate=0.2))
model3.add(Dense(50))
model3.add(Dense(50))
model3.add(Dense(vocab_size, activation='softmax'))
print(model3.summary())
# compile model
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history=model3.fit(X, y, epochs=100)

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_44 (LSTM)               (None, 100)               55600     
_________________________________________________________________
dropout_21 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_39 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_40 (Dense)             (None, 38)                1938      
Total params: 65,138
Trainable params: 65,138
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
12/12 [==============================] - 0s 7ms/step - loss: 3.5793 - accuracy: 0.1068
Epoch 2/100

In [129]:
# save the model to file
model3.save('model3.h5')

In [130]:
# test start of rhyme
print(generate_seq(model3, mapping, 10, 'Sing a son', 20))
# test mid-line
print(generate_seq(model3, mapping, 10, 'king was i', 20))
# test not in original
print(generate_seq(model3, mapping, 10, 'hello worl', 20))

Sing a song of sixpence,A pock
king was in his counting house
hello worle.Fulr tindgd.To tth


**The model3 also performs well on the first two cases, still not well on the third case. However, the prediction on the third case looks a little more like real words. The overfit may has improved a little bit.**

In [55]:
# 2. Split the text corpus file into training and validation 

from sklearn.model_selection import train_test_split

# split the data into 80% train 20% validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=42)

In [131]:
# Define a new model based on model3, called model_new
# pipe in validation sets to evaluate accuracy
model_new = Sequential()
model_new.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model_new.add(Dropout(rate=0.2))
model_new.add(Dense(50))
model_new.add(Dense(50))
model_new.add(Dense(vocab_size, activation='softmax'))
print(model_new.summary())
# compile model
model_new.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
history=model_new.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_45 (LSTM)               (None, 100)               55600     
_________________________________________________________________
dropout_22 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_42 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_43 (Dense)             (None, 38)                1938      
Total params: 65,138
Trainable params: 65,138
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
10/10 [==============================] - 1s 53ms/step - loss: 3.6167 - accuracy: 0.0879 - val_loss

**3. Summary: 
It appears that the new model is overfit. From model1 to model3, they all perform pretty well on the texts that have been trained but very poorly on the text that have not been trained (validation set).**

**I have applied several methods, including: increase cells from 75 to 100 in LSTM layer, increase epochs in training from 50 to 100, add dropout layer with dropout rate at 0.2, add two fully-connected layers with 50 nodes in each.**

**My intutition is that increase LSTM cells and training epochs will improve performance, however, the model easily becomes overfit. I try to regulate the overfit issue by adding Dropout layer and two additional Dense layers, it has lower the accuracy for a little bit. When I tested with the model3 methods piped with validation set, the method does work and the model becomes less overfit (validation accuracy has incrased from 8% in model1 to 18% in model3). However, it is still overfit. I suspect the fundamental reason of overfit is due to the very small text corpus. I need a larger text corpus to train the model.**